In [1]:
from pyiceberg.catalog import load_catalog

warehouse_path = "../tmp/warehouse"
catalog = load_catalog(
    "default",
    **{
        'type': 'sql',
        "uri": f"sqlite:///{warehouse_path}/pyiceberg_catalog.db",
        "warehouse": f"file://{warehouse_path}",
    },
)

Get data

In [2]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o ../tmp/yellow_tripdata_2023-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  14.8M      0  0:00:03  0:00:03 --:--:-- 14.8M


Load data

In [3]:
import pyarrow.parquet as pq

df = pq.read_table("../tmp/yellow_tripdata_2023-01.parquet")

In [4]:
catalog.create_namespace("default")

Create a new iceberg table!

In [5]:
table = catalog.create_table(
    "default.taxi_dataset",
    schema=df.schema,
)

In [6]:
table.append(df)
len(table.scan().to_arrow())

3066766

3066766 rows have been written to the table.

Now generate a tip-per-mile feature to train the model on:

In [7]:
import pyarrow.compute as pc

df = df.append_column("tip_per_mile", pc.divide(df["tip_amount"], df["trip_distance"]))

Evolve the schema of the table with the new column:

In [8]:
with table.update_schema() as update_schema:
    update_schema.union_by_name(df.schema)

And now we can write the new dataframe to the Iceberg table:

In [9]:
table.overwrite(df)
print(table.scan().to_arrow())

pyarrow.Table
VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: large_string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
tip_per_mile: double
----
VendorID: [[2,2,2,1,2,...,2,2,1,1,1],[1,2,2,2,2,...,1,1,1,2,2],...,[2,2,2,2,2,...,2,2,2,2,2],[2,2,2,2,2,...,2,2,2,2,2]]
tpep_pickup_datetime: [[2023-01-01 00:32:10.000000,2023-01-01 00:55:08.000000,2023-01-01 00:25:04.000000,2023-01-01 00:03:48.000000,2023-01-01 00:10:29.000000,...,2023-01-02 21:16:11.000000,2023-01-02 21:56:02.000000,2023-01-02 21:04:31.000000,2023-01-02 21:13:09.000000,2023-01-02 21:45:30.000000],[2023-01-02 21:49:54.000000,2023-01-02 21:17:06.000000,2023-01-02 21:35:06.000000,2023-01-02 21:18:43.00

Let's see the number of rows with non-zero tips:

In [10]:
df = table.scan(row_filter="tip_per_mile > 0").to_arrow()
len(df)

2371784

## Explore Iceberg data and metadata files
Since the catalog was configured to use the local filesystem, we can explore how Iceberg saved data and metadata files from the above operations.

In [11]:
!find ../tmp/warehouse/

../tmp/warehouse/
../tmp/warehouse/pyiceberg_catalog.db
../tmp/warehouse/default.db
../tmp/warehouse/default.db/taxi_dataset
../tmp/warehouse/default.db/taxi_dataset/data
../tmp/warehouse/default.db/taxi_dataset/data/00000-0-e3c00619-b933-4872-ba52-9bb79f28fc53.parquet
../tmp/warehouse/default.db/taxi_dataset/data/00000-0-acfe0b18-5b2a-4007-9dc7-3b82cb4437c3.parquet
../tmp/warehouse/default.db/taxi_dataset/metadata
../tmp/warehouse/default.db/taxi_dataset/metadata/00000-a6b92b85-3262-401c-a048-7b635f5c8874.metadata.json
../tmp/warehouse/default.db/taxi_dataset/metadata/acfe0b18-5b2a-4007-9dc7-3b82cb4437c3-m0.avro
../tmp/warehouse/default.db/taxi_dataset/metadata/00001-b5b2dbb1-8937-4c06-be5c-957bfc959dea.metadata.json
../tmp/warehouse/default.db/taxi_dataset/metadata/snap-4853850003843113212-0-e3c00619-b933-4872-ba52-9bb79f28fc53.avro
../tmp/warehouse/default.db/taxi_dataset/metadata/snap-8113952059566505016-0-acfe0b18-5b2a-4007-9dc7-3b82cb4437c3.avro
../tmp/warehouse/default.db/taxi_d